<a href="https://colab.research.google.com/github/meatorange/big_data_labs/blob/main/6408_%D0%96%D0%BE%D0%B2%D1%82%D1%8B%D0%B9%D0%90%D0%92_MapReduceExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator, Tuple
import numpy as np

np.random.seed(0)
mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates: Tuple[int, int], value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.411649429368285),
 (1, 2.411649429368285),
 (2, 2.411649429368285),
 (3, 2.411649429368285),
 (4, 2.411649429368285)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network # OK
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('what', 10)]),
 (1, [('is', 18), ('it', 18)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.02021839744032572),
   (None, 0.07103605819788694),
   (None, 0.08712929970154071),
   (None, 0.11827442586893322),
   (None, 0.1433532874090464),
   (None, 0.26455561210462697),
   (None, 0.3834415188257777),
   (None, 0.4146619399905236),
   (None, 0.4236547993389047),
   (None, 0.4375872112626925),
   (None, 0.45615033221654855),
   (None, 0.46147936225293185)]),
 (1,
  [(None, 0.5218483217500717),
   (None, 0.5288949197529045),
   (None, 0.5680445610939323),
   (None, 0.5684339488686485),
   (None, 0.6399210213275238),
   (None, 0.6458941130666561),
   (None, 0.7742336894342167),
   (None, 0.7781567509498505),
   (None, 0.7805291762864555),
   (None, 0.7917250380826646),
   (None, 0.7991585642167236),
   (None, 0.832619845547938),
   (None, 0.8700121482468192),
   (None, 0.8917730007820798),
   (None, 0.925596638292661),
   (None, 0.9446689170495839),
   (None, 0.9636627605010293),
   (None, 0.978618342232764)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
from typing import List

size = 1000

def MAP(_, value):
    yield (1, value)

def REDUCE(_, row):
    yield max(row)

def RECORDREADER():
    return [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=size))]

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(output)

[999]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
size = 1000

def MAP(_, value):
    yield (1, value)

def REDUCE(_, row):
    yield sum(row) / len(row)

def RECORDREADER():
    return [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=size))]

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(output)


[495.957]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
size = 10

def RECORDREADER():
    return [(index, num) for (index, num) in enumerate(np.random.randint(1000, size=size))]

def groupbykey_sorted(iterable):
    t = {}

    for (k2, v2) in iterable:
        t[k2] = sorted(t.get(k2, []) + [v2])

    return t.items()

shuffle_output = groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))
sorted_shuffle_output = groupbykey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER())))
shuffle_output, sorted_shuffle_output

(dict_items([(1, [608, 347, 804, 358, 404, 383, 492, 387, 658, 205])]),
 dict_items([(1, [51, 231, 268, 481, 543, 561, 623, 658, 866, 987])]))

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
size = 100

def MAP(_, value):
    yield (value, value)

def REDUCE(key, _):
    yield key

def RECORDREADER():
    return [(index, value) for (index, value) in enumerate(np.random.randint(10, size=size))]

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(sorted(output))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
# пример на словах
d1 = "it is what it is " * 3
d2 = "what is it " * 2
d3 = "it is a banana "
documents = [d1, d2, d3] * 2

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents) / maps))

    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i : i + split_size])

def MAP(docId, line):
    for word in line.split(" "):
        yield (word, 1)

def REDUCE(word, _):
    yield word

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output



56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'what']), (1, ['is', 'it'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
input_collection_ = input_collection.copy()
input_collection_.append(User(id=5, age=19, gender='male', social_contacts=444))
print(*input_collection_, sep='\n', end='\n\n')

def MAP(_, value):
    if value.age > 20:
        yield (value, value)

def REDUCE(*x):
    yield x

def RECORDREADER():
    return [(u.id, u) for u in input_collection_]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(map(lambda x: x[0], list(sorted(output))))
output


User(id=0, age=55, social_contacts=20, gender='male')
User(id=1, age=25, social_contacts=240, gender='female')
User(id=2, age=25, social_contacts=500, gender='female')
User(id=3, age=33, social_contacts=800, gender='female')
User(id=5, age=19, social_contacts=444, gender='male')



[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=2, age=25, social_contacts=500, gender='female'),
 User(id=3, age=33, social_contacts=800, gender='female')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
class UserProjection(NamedTuple):
    id: int
    age: int

input_collection_ = input_collection.copy()
input_collection_.extend([
    User(id=5, age=19, gender='male', social_contacts=444),
    User(id=5, age=19, gender='male', social_contacts=444)
])

def MAP(_, value):
    yield (UserProjection(id=value.id, age=value.age), UserProjection(id=value.id, age=value.age))

def REDUCE(key, _):
    yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in input_collection_]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output


[(UserProjection(id=0, age=55), UserProjection(id=0, age=55)),
 (UserProjection(id=1, age=25), UserProjection(id=1, age=25)),
 (UserProjection(id=2, age=25), UserProjection(id=2, age=25)),
 (UserProjection(id=3, age=33), UserProjection(id=3, age=33)),
 (UserProjection(id=5, age=19), UserProjection(id=5, age=19))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
first = input_collection.copy()
second = [first[-1], User(id=5, age=19, gender='male', social_contacts=444)]

def MAP(_, value):
    yield (value, value)

def REDUCE(key, _):
    yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in first + second]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output


[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='female'),
  User(id=2, age=25, social_contacts=500, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=5, age=19, social_contacts=444, gender='male'),
  User(id=5, age=19, social_contacts=444, gender='male'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
def REDUCE(key, rows):
    if len(rows) > 1:
        yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in first + second]

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(sorted(output))


[(User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
second = [first[-1], first[-2]]

def MAP(key, value):
    def contains(list, item):
        for el in list:
            if el is item:
                return True
        return False

    if contains(first, value):
        yield (value, 0)
    elif contains(second, value):
        yield (value, 1)

def REDUCE(key, rows):
    if (rows == [0]):
        yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in first + second]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output


[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
class NumberPlate(NamedTuple):
    number_plate: str
    owner: str

class PartialNumberPlate(NamedTuple):
    owner: str

number_plates = [
    NumberPlate(number_plate="А001АА 777", owner="Vladimir"),
    NumberPlate(number_plate="У753РС 78", owner="Ivan"),
    NumberPlate(number_plate="Е034МВ 145", owner="Leonid"),
    NumberPlate(number_plate="УВ3467 163", owner="Fast LTD")
]

class Car(NamedTuple):
    car_brand: str
    model: str
    number_plate: str

class PartialCar(NamedTuple):
    car_brand: str
    model: str

cars = [
    Car(car_brand="Aurus", model="Senat", number_plate="А001АА 777"),
    Car(car_brand="Lada", model="2107", number_plate="У753РС 78"),
    Car(car_brand="BMW", model="320i", number_plate="Е034МВ 145")
]

class CarNumberPlate(NamedTuple):
    car_brand: str
    model: str
    owner: str

def RECORDREADER():
    return [(index, u) for index, u in enumerate(number_plates + cars)]

def MAP(_, value):
    if value in cars:
        yield (value.number_plate, (0, PartialCar(value.car_brand, value.model)))
    elif value in number_plates:
        yield (value.number_plate, (1, PartialNumberPlate(value.owner)))

def REDUCE(key, rows):
  for i in range(len(rows)):
    for j in range(len(rows)):
      if (rows[i][0] == 1 and rows[j][0] == 0):
        yield (key, CarNumberPlate(rows[j][1].car_brand,
                                   rows[j][1].model,
                                   rows[i][1].owner))

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(sorted(output))


[('А001АА 777',
  CarNumberPlate(car_brand='Aurus', model='Senat', owner='Vladimir')),
 ('Е034МВ 145', CarNumberPlate(car_brand='BMW', model='320i', owner='Leonid')),
 ('У753РС 78', CarNumberPlate(car_brand='Lada', model='2107', owner='Ivan'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
class Car(NamedTuple):
    car_brand: str
    model: str
    price: int

cars = [
    Car("Lada", "2107", 20_000),
    Car("Lada", "Granta", 1_200_000),
    Car("Lada", "Vesta", 2_350_000),
    Car("BMW", "320i", 5_600_000),
    Car("BMW", "M340i", 10_000_000),
    Car("KIA", "Optima", 2_125_000)
]

def MAP(_, value):
    yield (value.car_brand, value.price)

def REDUCE(key, rows):
    yield (key, sum(rows) / len(rows))

def RECORDREADER():
    return [(index, u) for index, u in enumerate(cars)]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(sorted(output))
output


[('BMW', 7800000.0), ('KIA', 2125000.0), ('Lada', 1190000.0)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
ROWS = 5
COLS = 4
STRIPES = 2
COLS_IN_STRIPE = COLS // STRIPES

M = [np.ones((ROWS, COLS_IN_STRIPE)) for i in range(STRIPES)]
v = [np.random.rand(COLS_IN_STRIPE) for i in range(STRIPES)]

def MAP(coordinates: Tuple[int, int], value: int):
    i, j = coordinates
    yield (i, value * vec[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)

def RECORDREADER():
    global vec
    for stripe_index, stripe in enumerate(M):
        vec = v[stripe_index]
        for i in range(stripe.shape[0]):
            for j in range(stripe.shape[1]):
                yield ((i, j), stripe[i, j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
list(output)


[(0, 1.0670525178683352),
 (1, 1.0670525178683352),
 (2, 1.0670525178683352),
 (3, 1.0670525178683352),
 (4, 1.0670525178683352)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
    (j, k) = k1
    w = v1
    # solution code that yield(k2,v2) pairs # OK
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
    (i, k) = key
    # solution code that yield(k3,v3) pairs # OK
    yield ((i, k), sum(values))


Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output)+1
    K = max(k for ((i,k), vw) in reduce_output)+1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
    for i in range(I):
        for j in range(J):
            yield ((0, i, j), small_mat[i, j])

    for j in range(J):
        for k in range(K):
            yield ((1, j, k), big_mat[j, k])

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1
    if matrix_index == 0:
        yield (col_index, (matrix_index, row_index, v1))
    else:
        yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
    first_matrix = [el for el in values if el[0] == 0]
    second_matrix = [el for el in values if el[0] == 1]

    for first_el in first_matrix:
        for second_el in second_matrix:
            (_, first_index, first_value) = first_el
            (_, second_index, second_value) = second_el
            yield ((first_index, second_index), first_value * second_value)


def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_RECORDREADER(iterable):
    def func():
        for el in iterable:
            yield el
    return func


reference_solution = np.matmul(small_mat, big_mat)
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    def RECORDREADER(index, matrix):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                yield ((index, i, j), matrix[i, j])

    yield RECORDREADER(0, small_mat)
    yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
    (matrix_index, row_index, col_index) = k1

    if matrix_index == 0:
            yield (col_index, (matrix_index, row_index, v1))
    else:
            yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
    first_matrix = [el for el in values if el[0] == 0]
    second_matrix = [el for el in values if el[0] == 1]

    for first_el in first_matrix:
        for second_el in second_matrix:
            (_, first_index, first_value) = first_el
            (_, second_index, second_value) = second_el
            yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
    def func():
        for el in iterable:
            yield el[1]
    return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
    for v in value[1]:
        solution_unpacked.append(v)

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true


126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    def RECORDREADER(index, matrix, shift):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                yield ((index, i + shift, j), matrix[i, j])

    split_size = int(np.ceil(len(small_mat) / maps))

    for i in range(0, len(small_mat), split_size):
        yield RECORDREADER(0, small_mat[i:i + split_size], i)

    split_size = int(np.ceil(len(big_mat) / maps))

    for i in range(0, len(big_mat), split_size):
        yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
    first_matrix = [el for el in values if el[0] == 0]
    second_matrix = [el for el in values if el[0] == 1]

    for first_el in first_matrix:
        for second_el in second_matrix:
            (_, first_index, first_value) = first_el
            (_, second_index, second_value) = second_el
            yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
    yield args

def REDUCE_AGREGATE(key, values):
    yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
    def func():
        for el in iterable:
            yield el[1]
    return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
    for v in value[1]:
        solution_unpacked.append(v)

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true


18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True